<a href="https://colab.research.google.com/github/etodasha/PRA-M2.875-2020.2/blob/main/PRA_DL_InceptionResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [ ]:
!kaggle datasets download "jordidelatorreuoc/kaggle-plant-pathology-2021-modificat"

 98% 955M/970M [00:05<00:00, 129MB/s]
100% 970M/970M [00:05<00:00, 170MB/s]


In [ ]:
!unzip kaggle-plant-pathology-2021-modificat.zip

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import keras 
import os
import matplotlib.pylab as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, ZeroPadding2D, GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam, SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from keras.layers.experimental.preprocessing import Resizing
from keras.callbacks import *
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Model

DATA AUGMENTATION & TRAIN VAL SPLIT

In [ ]:
datagen_train = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=[0.5,1.2],
    rotation_range=20,
    rescale=1./255,
    validation_split=0.2) 

train_generator = datagen_train.flow_from_directory( 
    'train',
    subset='training')

validation_generator = datagen_train.flow_from_directory(
    'train',
    subset='validation'
    ,shuffle=False)



Found 7800 images belonging to 6 classes.
Found 1950 images belonging to 6 classes.


In [ ]:
from keras import backend as K

def custom_recall(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = TP / (Positives+K.epsilon())
  return recall 

def custom_precision(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = TP / (Pred_Positives+K.epsilon())
  return precision

def custom_f1(y_true, y_pred):    
    precision, recall = custom_precision(y_true, y_pred), custom_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


TRANSFER LEARNING + FINE TUNING

InceptionResNetV2

In [ ]:
############
# InceptionResNetV2 #
#############

inceptionResNet = InceptionResNetV2(weights='imagenet', include_top=False)
inceptionResNet.trainable = False 
inceptionResNet.summary()

219070464/219055592 [==============================] - 1s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, None, None, 3 96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization_4[0][0]      
_____________

In [ ]:
from keras.applications.inception_resnet_v2 import preprocess_input

inputs = keras.Input(shape=(332, 498, 3))
resize = Resizing(299,299)(inputs)

preprocess = preprocess_input(resize)

my_inceptionResNet = inceptionResNet(preprocess, training=False)

my_inceptionResNet = GlobalAveragePooling2D()(my_inceptionResNet)
my_inceptionResNet = Dense(1000)(my_inceptionResNet)
my_inceptionResNet = Dropout(0.5)(my_inceptionResNet)
my_inceptionResNet = Dense(6, activation="softmax")(my_inceptionResNet)

my_inceptionResNet_model = Model(inputs=inputs, outputs=my_inceptionResNet)

my_inceptionResNet_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 55,879,742
Trainable params: 1,543,006
Non-trainable params: 54,336,736
_________________________________________________________________


In [ ]:
# fine tuning 

inceptionResNet.trainable = True

my_inceptionResNet_model.compile(optimizer=Adam(learning_rate=1e-5),  # Very low learning rate to fine tune
              loss='categorical_crossentropy',
              metrics=['accuracy',custom_f1,custom_recall,custom_precision])

filepath = "/content/gdrive/My Drive/InceptionResNet_epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"
checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)



early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=10, 
                                               restore_best_weights=True, verbose=2)

inceptionResNet_history = my_inceptionResNet_model.fit(train_generator,
                                         validation_data = validation_generator,
                                         epochs=50,
                                         verbose=1,
                                         batch_size=32,
                                         callbacks=[early_stopping,checkpoint_callback])

Epoch 1/50
244/244 [==============================] - 353s 1s/step - loss: 1.6316 - accuracy: 0.4044 - custom_f1: 0.0386 - custom_recall: 0.0286 - custom_precision: 0.1279 - val_loss: 1.5966 - val_accuracy: 0.4103 - val_custom_f1: 0.1231 - val_custom_recall: 0.0743 - val_custom_precision: 0.3934
Epoch 2/50
244/244 [==============================] - 318s 1s/step - loss: 1.5655 - accuracy: 0.4176 - custom_f1: 0.2116 - custom_recall: 0.1368 - custom_precision: 0.5850 - val_loss: 1.5009 - val_accuracy: 0.4369 - val_custom_f1: 0.2104 - val_custom_recall: 0.1434 - val_custom_precision: 0.4262
Epoch 3/50
244/244 [==============================] - 318s 1s/step - loss: 1.4323 - accuracy: 0.4644 - custom_f1: 0.3518 - custom_recall: 0.2493 - custom_precision: 0.6699 - val_loss: 1.2996 - val_accuracy: 0.5297 - val_custom_f1: 0.4308 - val_custom_recall: 0.3663 - val_custom_precision: 0.5313
Epoch 4/50
244/244 [==============================] - 319s 1s/step - loss: 1.2199 - accuracy: 0.5592 - custom

In [ ]:
# IncetionResNet 0.8377